In [103]:
import Bio
import pandas as pd

from itertools import combinations
from Bio import Entrez, SeqIO, Align, AlignIO

## Part 1.

### esearch

In [7]:
Entrez.email = 'riserisen@yandex.ru'

In [19]:
handle = Entrez.esearch(db="nucleotide", term='"Homo sapiens"[Organism] AND  GSPT1[Gene]')
record = handle.read()
print(record)

<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">
<eSearchResult><Count>9</Count><RetMax>9</RetMax><RetStart>0</RetStart><IdList>
<Id>1676355513</Id>
<Id>1676319656</Id>
<Id>1519312966</Id>
<Id>568815582</Id>
<Id>74273666</Id>
<Id>74230029</Id>
<Id>33874733</Id>
<Id>39754980</Id>
<Id>307685420</Id>
</IdList><TranslationSet><Translation>     <From>"Homo sapiens"[Organism]</From>     <To>"Homo sapiens"[Organism]</To>    </Translation></TranslationSet><TranslationStack>   <TermSet>    <Term>"Homo sapiens"[Organism]</Term>    <Field>Organism</Field>    <Count>27614182</Count>    <Explode>Y</Explode>   </TermSet>   <TermSet>    <Term>GSPT1[Gene]</Term>    <Field>Gene</Field>    <Count>1020</Count>    <Explode>N</Explode>   </TermSet>   <OP>AND</OP>  </TranslationStack><QueryTranslation>"Homo sapiens"[Organism] AND GSPT1[Gene]</QueryTranslation></eSearchResult>



### esearch + esummary

In [43]:
handle = Entrez.esearch(db="nucleotide", term='"Homo sapiens"[Organism] AND  GSPT1[Gene]')
record = Entrez.read(handle)
id_list, caption_list, length_list = [], [], []

summary_handle = Entrez.esummary(db="nucleotide", id=','.join(record["IdList"]))
summary = Entrez.read(summary_handle)
for rec in summary:
    id_list.append(rec['Id'])
    caption_list.append(rec['Caption'])
    length_list.append(rec['Length'])

df = pd.DataFrame({'UID': id_list, 'Accession_number': caption_list, 'Sequence length': length_list})
df

,UID,Accession_number,Sequence length
0,1676355513,NM_001130006,7138
1,1676319656,NM_001130007,7166
2,1519312966,NM_002094,7141
3,568815582,NC_000016,90338345
4,74273666,CM000267,75226909
5,74230029,CH471112,14690834
6,33874733,BC009503,2523
7,39754980,AY398991,1562
8,307685420,AB590486,1919


### esearch + efetch

In [51]:
handle = Entrez.esearch(db="nucleotide", term='"Homo sapiens"[Organism] AND  GSPT1[Gene]')
record = Entrez.read(handle)

with open('fetched_sequences.fa', 'w') as f:
    fetch_handle = Entrez.efetch(db="nucleotide", id=','.join(record["IdList"][:3]), rettype="fasta", retmode="text")
    f.write(fetch_handle.read())

In [55]:
with open('fetched_sequences.fa', 'r') as f:
    print(f.read()[:1111])

>NM_001130006.2 Homo sapiens G1 to S phase transition 1 (GSPT1), transcript variant 2, mRNA
GCTCACACACACGAGGAGGAGGGTTGAGCTGCCGCCGCCGCCGCCTCTGTCGTCGTCGCGAGTGTGGAGT
CGGGACTGGAGCTGCTGCCGCGGCGACGCCGGGGATCTTTGTCGCTAGCTCCCGGCCCTTCTGCCCCGCC
GCCTTCCCTCAGTCAGCGTTGCCCACTCCTCTCCGGCCGGGCGCCCCTGCCTCCATTTCCCGCTCTCTGT
CCACCACACACACGGCCCCCCCGATCATGGATCCGGGCAGTGGCGGCGGCGGCGGCGGCGGCGGCGGCGG
CGGGAGCAGCAGCGGCAGCAGCAGCAGCGACTCGGCGCCTGACTGCTGGGACCAGGCGGACATGGAAGCC
CCCGGGCCGGGCCCTTGCGGCGGCGGCGGCTCCCTGGCGGCGGCGGCCGAGGCCCAGCGGGAGAACCTCA
GCGCGGCCTTCAGCCGGCAACTCAACGTCAACGCCAAGCCCTTCGTGCCCAACGTCCACGCCGCCGAGTT
CGTGCCGTCCTTCCTGCGGGGCCCGGCAGCGCCGCCACCCCCAGTTGGCGGCGCCGCCAATAACCACGGA
GCCGGCAGCGGCGCGGGAGGCCGTGCGGCACCTGTGGAATCCTCTCAAGAGGAACAGTCATTGTGTGAAG
GTTCAAATTCAGCTGTTAGCATGGAACTTTCAGAACCTATTGAAAATGGAGAGACAGAAATGTCTCCAGA
AGAATCATGGGAGCACAAAGAAGAAATAAGTGAAGCAGAGCCAGGGGGTGGTTCCTTGGGAGATGGAAGG
CCGCCAGAGGAAAGTGCCCATGAAATGATGGAGGAGGAAGAGGAAATCCCAAAACCTAAGTCTGTGGTTG
CACCGCCAGGTGCTCCTAAGAAAGAGCATGTAAATGTAGTATTCATTGGGCACGTA

### elink + efetch

In [88]:
pmid = '12890024'
paper = Entrez.elink(dbfrom="pubmed", db="nucleotide", id=pmid)
res = Entrez.read(paper)[0]['LinkSetDb']
print(res)
    
with open('linked_sequences.fa', 'w') as f:
    for link_set in res:
        db = link_set['DbTo']
        ids = ','.join([item['Id'] for item in link_set['Link']])
        fetch_handle = Entrez.efetch(db=db, id=ids, rettype="fasta", retmode="text")
        f.write(fetch_handle.read() + '\n')
    

[{'Link': [{'Id': '19568061'}, {'Id': '19568059'}, {'Id': '19568057'}, {'Id': '19568055'}, {'Id': '19568053'}, {'Id': '19568051'}, {'Id': '19568049'}, {'Id': '19568047'}, {'Id': '19568045'}, {'Id': '19568043'}, {'Id': '19568041'}, {'Id': '19568039'}, {'Id': '19568037'}, {'Id': '19568035'}, {'Id': '19568033'}, {'Id': '19568031'}, {'Id': '19568029'}, {'Id': '19568027'}, {'Id': '19568025'}, {'Id': '19568023'}, {'Id': '19568021'}, {'Id': '19568019'}, {'Id': '19568017'}, {'Id': '19568015'}, {'Id': '19567985'}, {'Id': '19567983'}, {'Id': '19567981'}, {'Id': '19567979'}, {'Id': '19567977'}, {'Id': '19567975'}, {'Id': '19567973'}, {'Id': '19567971'}, {'Id': '19567969'}, {'Id': '19567967'}, {'Id': '19567965'}, {'Id': '19567963'}, {'Id': '19567961'}, {'Id': '19567959'}, {'Id': '19567957'}, {'Id': '19567955'}], 'DbTo': 'nuccore', 'LinkName': 'pubmed_nuccore'}]


In [89]:
with open('linked_sequences.fa', 'r') as f:
    print(f.read()[:1111])

>AY028697.1 Saccharomyces cerevisiae strain SCI9 prion protein (URE2) gene, partial cds
TGCTGCAAATTAAGTTGTACACCAAATGATGAATAACAACGGCAACCAAGTGTCGAATCTCTCCAATGCG
CTCCGTCAAGTAAACATAGGAAACAGGAACAGTAATACAACCACCGATCAAAGTAATATAAATTTTGAAT
TTTCAACAGGTGTAAATAATAATAATAATAACAATAGCAGTAGTAATAACAATAATGTTCAAAACAATAA
CAGCGGCCGCAATGGTAGCCAAAATAATGATAACGAGAATAATATCAAGAATACCTTAGAACAACATCGA
CAACAACAACAGGCATTTTCGGATATGAGTCACGTGG

>AY028696.1 Saccharomyces cerevisiae strain SCI16 prion protein (URE2) gene, partial cds
TGCTGCAAATTAAGTTGTACACCAAATGATGAATAACAACGGCAACCAAGTGTCGAATCTCTCCAATGCG
CTCCGTCAAGTAAACATAGGAAACAGGAACAGTAATACAACCACCGATCAAAGTAATATAAATTTTGAAT
TTTCAACAGGTGTAAATAATAATAATAATAACAATAGCAGTAGTAATAACAATAATGTTCAAAACAATAA
CAGCGGCCGCAATGGTAGCCAAAATAATGATAACGAGAATAATATCAAGAATACCTTAGAACAACATCGA
CAACAACAACAGGCATTTTCGGATATGAGTCACGTGG

>AY028695.1 Saccharomyces cerevisiae strain SCI11 prion protein (URE2) gene, partial cds
TGCTGCAAATTAAGTTGTACACCAAATGATGAATAACAACGGCAACCAAGTGTCGAATCTCTCCAATGCG
CTCCGTCAAGTAAACAT

## Part 2.

### 1 Blast results
Organism - Kluyveromyces lactis  
LOCUS - CP021242  
type - genomic DNA  

### 2 Alignments  
COMMANDS:  
muscle -in SUP35_10seqs.fa -verbose -quiet -fasta -out muscle_aln.fa  
mafft --thread 2 --anysymbol --bl 62 --op 1.53 --ep 0.123 --reorder --retree 2 --treeout --maxiterate 2 --nuc SUP35_10seqs.fa  
t_coffee -in SUP35_10seqs.fa -case=upper -n_core=2 -output="fasta" -outorder=aligned -type=dna  
kalign -i SUP35_10seqs.fa -f fasta -s 80.0 -e 3.0 -t 3.0 -m 0.0

In [130]:
aligner = Align.PairwiseAligner()
aligners = ['mafft', 'muscle', 'tcoffee', 'kalign']
scores = {alnr: 0 for alnr in aligners}
aln_files = {alnr: f'data/sup35/almt/{alnr}_aln.fa' for alnr in aligners}
lengths = {}

for alnr in aligners:
    file = aln_files[alnr]
    align = AlignIO.read(file, "fasta")
    l = align.get_alignment_length()
    lengths[alnr] = l
    for i in range(l):
        if len(set(align[:, i])) == 1:
            scores[alnr] += 1

print("alignment lengths: ", lengths)
print("scores: ", scores)

alignment lengths:  {'mafft': 2286, 'muscle': 2275, 'tcoffee': 2210, 'kalign': 2500}
scores:  {'mafft': 1014, 'muscle': 1042, 'tcoffee': 1032, 'kalign': 1073}


|ALIGNER| RUNNING TIME| FULL MATCHES | GBLOCKS POSITIONS | GBLOCKS % |
|------|-----|------|------|------------------------------------|
|MUSCLE| 8s  | 1042 | 1753 | 77% of the original 2275 positions |
|MAFFT | 5s  | 1014 | 1786 | 78% of the original 2286 positions |
|TCOFFE| 40s | 1032 | 1722 | 77% of the original 2210 positions |
|KALIGN| 3s  | 1073 | 1377 | 55% of the original 2500 positions |

### 3 Faulty alignment
Sequence  
>SUP35_Spar_A12_Liti_  

should be reverse-complemented to fix the alignment.

### 4 Amino acid alignment

In [133]:
aligner = Align.PairwiseAligner()
aligners = ['mafft', 'muscle', 'tcoffee', 'kalign']
scores = {alnr: 0 for alnr in aligners}
aln_files = {alnr: f'data/sup35/almt/{alnr}_amin_aln.fa' for alnr in aligners}
lengths = {}

for alnr in aligners:
    file = aln_files[alnr]
    align = AlignIO.read(file, "fasta")
    l = align.get_alignment_length()
    lengths[alnr] = l
    for i in range(l):
        if len(set(align[:, i])) == 1:
            scores[alnr] += 1

print("alignment lengths: ", lengths)
print("scores: ", scores)

alignment lengths:  {'mafft': 755, 'muscle': 743, 'tcoffee': 752, 'kalign': 793}
scores:  {'mafft': 412, 'muscle': 407, 'tcoffee': 410, 'kalign': 427}


|ALIGNER| RUNNING TIME| FULL MATCHES | GBLOCKS POSITIONS | GBLOCKS % |
|------|-----|-----|-----|-----------------------------------|
|MUSCLE| 2s  | 407 | 553 | 74% of the original 743 positions |
|MAFFT | 2s  | 412 | 542 | 71% of the original 755 positions |
|TCOFFE| 7s  | 410 | 558 | 74% of the original 752 positions |
|KALIGN| 3s  | 427 | 480 | 60% of the original 793 positions |

## Results
Choice of the algorithm depends mostly on metric, you are mosted interested in.  
Tcoffee works slower in most cases.  
Kalign is fastest, but less accurate, since it maximazes number of full matches by increasing the number of indels.  
Muscle is the most balanced and preferable choice.